In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np

In [2]:
from pathlib import Path

root = "https://ai.stanford.edu/~amaas/data/sentiment/"
filename = "aclImdb_v1.tar.gz"
filepath = tf.keras.utils.get_file(filename, root + filename, extract=True, cache_dir="./imdb")
path = Path(filepath).with_name("aclImdb")
path

WindowsPath('/tmp/.keras/datasets/aclImdb')

In [3]:
def review_paths(dirpath):
    return [str(path) for path in dirpath.glob("*.txt")]

train_pos = review_paths(path / "train" / "pos")
train_neg = review_paths(path / "train" / "neg")
test_val_pos = review_paths(path / "test" / "pos")
test_val_neg = review_paths(path / "test" / "neg")

len(train_pos), len(train_neg), len(test_val_pos), len(test_val_neg)

(12500, 12500, 12500, 12500)

In [4]:
np.random.shuffle(test_val_pos)

test_pos = test_val_pos[:5000]
test_neg = test_val_neg[:5000]
valid_pos = test_val_pos[5000:]
valid_neg = test_val_neg[5000:]

In [5]:
train_pos[150]

'\\tmp\\.keras\\datasets\\aclImdb\\train\\pos\\10135_7.txt'

In [6]:
def imdb_dataset(filepaths_positive, filepaths_negative):
    reviews = []
    labels = []
    for filepaths, label in ((filepaths_negative, 0), (filepaths_positive, 1)):
        for filepath in filepaths:
            with open(filepath, encoding="utf8") as review_file:
                reviews.append(review_file.read())
            labels.append(label)
    return tf.data.Dataset.from_tensor_slices((tf.constant(reviews), tf.constant(labels)))

In [7]:
batch_size = 32

train_set = imdb_dataset(train_pos, train_neg).shuffle(25000, seed=42)
train_set = train_set.batch(batch_size).prefetch(1)
valid_set = imdb_dataset(valid_pos, valid_neg).batch(batch_size).prefetch(1)
test_set = imdb_dataset(test_pos, test_neg).batch(batch_size).prefetch(1)

In [26]:
max_tokens = 1000
sample_reviews = train_set.map(lambda review, label: review)
text_vectorization = tf.keras.layers.TextVectorization(max_tokens=max_tokens, output_mode="tf_idf")
text_vectorization.adapt(sample_reviews)

In [29]:
text_vectorization.get_vocabulary(0)[:10]

['the', 'and', 'a', 'of', 'to', 'is', 'in', 'it', 'i', 'this']

In [30]:
tf.random.set_seed(42)
model = tf.keras.Sequential([ 
    text_vectorization,
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid"),
])
model.compile(loss="binary_crossentropy", optimizer="nadam", metrics=["accuracy"])
model.fit(train_set, epochs=5, validation_data=valid_set)

Epoch 1/5
782/782 [==============================] - 7s 6ms/step - loss: 0.4488 - accuracy: 0.8191 - val_loss: 0.3670 - val_accuracy: 0.8460
Epoch 2/5
782/782 [==============================] - 5s 6ms/step - loss: 0.3667 - accuracy: 0.8529 - val_loss: 0.4119 - val_accuracy: 0.8369
Epoch 3/5
782/782 [==============================] - 5s 6ms/step - loss: 0.3261 - accuracy: 0.8692 - val_loss: 0.3544 - val_accuracy: 0.8503
Epoch 4/5
782/782 [==============================] - 5s 6ms/step - loss: 0.2715 - accuracy: 0.8900 - val_loss: 0.3705 - val_accuracy: 0.8459
Epoch 5/5
782/782 [==============================] - 5s 6ms/step - loss: 0.2149 - accuracy: 0.9164 - val_loss: 0.3969 - val_accuracy: 0.8391


In [33]:
def compute_mean_embedding(inputs):
    not_pad = tf.math.count_nonzero(inputs, axis=-1)
    n_words = tf.math.count_nonzero(not_pad, axis=-1, keepdims=True)    
    sqrt_n_words = tf.math.sqrt(tf.cast(n_words, tf.float32))
    return tf.reduce_sum(inputs, axis=1) / sqrt_n_words

In [34]:
embedding_size = 20
tf.random.set_seed(42)

text_vectorization = tf.keras.layers.TextVectorization(max_tokens=max_tokens, output_mode="int")
text_vectorization.adapt(sample_reviews)

model = tf.keras.Sequential([
    text_vectorization,
    tf.keras.layers.Embedding(input_dim=max_tokens,
                              output_dim=embedding_size,
                              mask_zero=True),  # <pad> tokens => zero vectors
    tf.keras.layers.Lambda(compute_mean_embedding),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid"),
])

In [35]:
model.compile(loss="binary_crossentropy", optimizer="nadam", metrics=["accuracy"])
model.fit(train_set, epochs=5, validation_data=valid_set)

Epoch 1/5
782/782 [==============================] - 9s 10ms/step - loss: 0.4653 - accuracy: 0.7773 - val_loss: 0.3637 - val_accuracy: 0.8453
Epoch 2/5
782/782 [==============================] - 8s 10ms/step - loss: 0.3312 - accuracy: 0.8587 - val_loss: 0.3312 - val_accuracy: 0.8549
Epoch 3/5
782/782 [==============================] - 8s 10ms/step - loss: 0.3163 - accuracy: 0.8637 - val_loss: 0.3378 - val_accuracy: 0.8512
Epoch 4/5
782/782 [==============================] - 8s 10ms/step - loss: 0.3098 - accuracy: 0.8680 - val_loss: 0.3218 - val_accuracy: 0.8590
Epoch 5/5
782/782 [==============================] - 8s 10ms/step - loss: 0.3081 - accuracy: 0.8691 - val_loss: 0.3382 - val_accuracy: 0.8512
